In [2]:
import pandas as pd

In [8]:
plans = pd.read_csv('../data/normalized_tables/plans.csv')
benefits = pd.read_csv('../data/normalized_tables/benefits.csv')
premium = pd.read_csv('../data/normalized_tables/premium.csv')
county_area = pd.read_csv('../data/normalized_tables/county_area_crosswalk.csv')
zip_fips = pd.read_csv('../data/normalized_tables/zip_fips_crosswalk.csv')
state_age = pd.read_csv('../data/normalized_tables/state_age_curve.csv')
deductibles = pd.read_csv('../data/normalized_tables/deductibles.csv')
moop = pd.read_csv('../data/normalized_tables/moop.csv')

In [10]:
benefits_bool_cols = [
    'LIMITED', 'CopayInnTier1Complex', 'CopayInnTier2Complex',
    'CoinsInnTier1Complex', 'CoinsInnTier2Complex',
    'CopayOutofNetComplex', 'CoinsOutofNetComplex'
]

for col in benefits_bool_cols:
    print(f"{col} unique values:", benefits[col].unique())

LIMITED unique values: [ 0. nan  1.]
CopayInnTier1Complex unique values: [ 0. nan  1.]
CopayInnTier2Complex unique values: [ 0. nan  1.]
CoinsInnTier1Complex unique values: [ 0. nan  1.]
CoinsInnTier2Complex unique values: [ 0. nan]
CopayOutofNetComplex unique values: [ 0. nan  1.]
CoinsOutofNetComplex unique values: [ 0. nan]


In [18]:
# Replace nan values by the code '99' for unknown and convert float to integer
for col in benefits_bool_cols:
    benefits[col] = pd.to_numeric(benefits[col], errors='coerce').fillna(99).astype(int)

In [16]:
for col in benefits_bool_cols:
    print(f"{col} unique values:", benefits[col].unique())

LIMITED unique values: [ 0 99  1]
CopayInnTier1Complex unique values: [ 0 99  1]
CopayInnTier2Complex unique values: [ 0 99  1]
CoinsInnTier1Complex unique values: [ 0 99  1]
CoinsInnTier2Complex unique values: [ 0 99]
CopayOutofNetComplex unique values: [ 0 99  1]
CoinsOutofNetComplex unique values: [ 0 99]


In [26]:
# The following columns are those which we need to be integer datatype in our database

benefits_int_cols = [
    'LIMITED', 'CopayInn_TIERS', 'CopayInnTier1Complex', 'CopayInnTier1',
    'CopayInnTier2Complex', 'CopayInnTier2',
    'CoinsInn_TIERS', 'CoinsInnTier1Complex', 'CoinsInnTier1',
    'CoinsInnTier2Complex', 'CoinsInnTier2',
    'CopayOutofNetComplex', 'CopayOutofNet',
    'CoinsOutofNetComplex', 'CoinsOutofNet'
]

for col in benefits_int_cols:
    print(f"{col} unique values:", benefits[col].unique())

LIMITED unique values: [ 0 99  1]
CopayInn_TIERS unique values: [ 1.  2. nan  3.]
CopayInnTier1Complex unique values: [ 0 99  1]
CopayInnTier1 unique values: [ 0.  3.  4.  2.  1. nan  6.  8.]
CopayInnTier2Complex unique values: [ 0 99  1]
CopayInnTier2 unique values: [ 0.  3.  2. nan  1.  4.  6.  8.]
CoinsInn_TIERS unique values: [ 1.  2. nan  3.]
CoinsInnTier1Complex unique values: [ 0 99  1]
CoinsInnTier1 unique values: [ 4.  3.  0. nan]
CoinsInnTier2Complex unique values: [ 0 99]
CoinsInnTier2 unique values: [ 0.  4.  3. nan]
CopayOutofNetComplex unique values: [ 0 99  1]
CopayOutofNet unique values: [ 0.  3.  4.  2.  1. nan  8.]
CoinsOutofNetComplex unique values: [ 0 99]
CoinsOutofNet unique values: [ 4.  3.  0. nan]


In [28]:
# Replace nan values by the code '99' for unknown and convert float to integers
for col in benefits_int_cols:
    benefits[col] = pd.to_numeric(benefits[col], errors='coerce').fillna(99).astype(int)

In [34]:
benefits.to_csv('../data/normalized_tables/benefits.csv', index=False)

In [36]:
# nan values are ok when we want the column datatype to be numeric
benefits.head()

,PLANID,AREA,LIMITED,CopayInn_TIERS,CopayInnTier1Complex,CopayInnTier1,CopayInnTier1A,CopayInnTier2Complex,CopayInnTier2,CopayInnTier2A,...,CoinsInnTier2Complex,CoinsInnTier2,CoinsInnTier2A,CopayOutofNetComplex,CopayOutofNet,CopayOutofNetA,CoinsOutofNetComplex,CoinsOutofNet,CoinsOutofNetA,benefit_code
0,73836AK0950001,AK01,0,1,0,0,NaN,0,0,NaN,...,0,0,NaN,0,0,NaN,0,4,40.0,AB
1,73836AK0950001,AK02,0,1,0,0,NaN,0,0,NaN,...,0,0,NaN,0,0,NaN,0,4,40.0,AB
2,73836AK0930001,AK02,0,1,0,0,NaN,0,0,NaN,...,0,0,NaN,0,0,NaN,0,4,30.0,AB
3,73836AK0950001,AK03,0,1,0,0,NaN,0,0,NaN,...,0,0,NaN,0,0,NaN,0,4,40.0,AB
4,73836AK0930001,AK03,0,1,0,0,NaN,0,0,NaN,...,0,0,NaN,0,0,NaN,0,4,30.0,AB


In [44]:
# Since we originally created PREMI21_BASE as a fraction of PREMI27/(multiplier for 27), it is stored as a long decimal

premium.head()

,PLANID,AREA,ST,PREMI21_BASE,PREMI2C30,PREMC2C30
0,73836AK0950001,AK01,AK,789.122137,2104.0,3000.0
1,73836AK0950001,AK02,AK,830.152672,2212.0,3154.0
2,73836AK0930001,AK02,AK,745.229008,1985.0,2830.0
3,73836AK0950001,AK03,AK,808.206107,2153.0,3070.0
4,73836AK0930001,AK03,AK,725.190840,1933.0,2756.0


In [46]:
# We want to round up PREMI21_BASE to 2 decimal places, as it is supposed to be dollar value

premium['PREMI21_BASE'] = premium['PREMI21_BASE'].round(2)
premium.to_csv('../data/normalized_tables/premium.csv', index=False)

In [54]:
# The Kaggle dataset for ZIP-COUNTY-FIP CROSSWALK seemed like it had all unique values for ZIP

zip_fips.head()

,ZIP,COUNTY,ST,FIPS
0,36003,Autauga County,AL,1001
1,36006,Autauga County,AL,1001
2,36067,Autauga County,AL,1001
3,36066,Autauga County,AL,1001
4,36703,Autauga County,AL,1001


In [84]:
# However, we have repeated values for ZIP, perhaps because one ZIP code could be linked to 2 counties.

for col in zip_fips.columns:
    col_duplicates = zip_fips[zip_fips[col].duplicated()]
    print(f"Number of duplicate {col}: {len(col_duplicates)}")

Number of duplicate ZIP: 13433
Number of duplicate COUNTY: 50931
Number of duplicate ST: 52835
Number of duplicate FIPS: 49666


In [64]:
# For example, let's look at all rows where ZIP = 36053

zip_fips[zip_fips['ZIP'] == 36053]

,ZIP,COUNTY,ST,FIPS
46,36053,Barbour County,AL,1005
85,36053,Bullock County,AL,1011


In [97]:
# We can use a composite key (ZIP, COUNTY) for the zip_fips table
# Let's verify that the combination is indeed unique.

is_repeated = zip_fips[['ZIP', 'COUNTY']].duplicated().any()
print("Is (ZIP, COUNTY) unique for all rows?:", not is_repeated)

Is (ZIP, COUNTY) unique for all rows?: True


In [72]:
county_area.head()

,FIPS,COUNTY,AREA_COUNT,AREA
0,1001,Autauga County,1,AL11
1,1003,Baldwin County,1,AL13
2,1005,Barbour County,1,AL13
3,1007,Bibb County,1,AL03
4,1009,Blount County,1,AL03


In [99]:
# The HIX COUNTY-RATING-AREA CROSSWALK seemed like it had all unique values for FIP

repeated_fips = county_area['FIPS'][county_area['FIPS'].duplicated()]
print(repeated_fips.unique())

[ 2013  2016  2020  2050  2063  2070  2122  2150  2158  2164  2290  6037
 25005 25007 25013 25017 25021 25023 25027]


In [86]:
# However, all of the columns have duplicate values, so we must choose a composite primary key for the county_area table

for col in county_area.columns:
    col_duplicates = county_area[county_area[col].duplicated()]
    print(f"Number of duplicate {col}: {len(col_duplicates)}")

Number of duplicate FIPS: 24
Number of duplicate COUNTY: 1289
Number of duplicate AREA_COUNT: 3163
Number of duplicate AREA: 2663


In [111]:
# We can use a composite key (AREA, COUNTY) for the county_area table
# Let's verify that the combination is indeed unique.

is_repeated = county_area[['AREA', 'COUNTY']].duplicated().any()
print("Is (AREA, COUNTY) unique for all rows?:", not is_repeated)

Is (AREA, COUNTY) unique for all rows?: True


In [115]:
# We see that all NaN values in the deductible table are in columns that either end with the string '_TIERS' or '_AMOUNT'

deductibles.isnull().sum()

PLANID                          0
AREA                            0
MED_IN_IND_CODE                 0
MED_IN_IND_TIERS            58444
MED_IN_IND_TIER1_AMOUNT     58444
MED_IN_IND_TIER2_AMOUNT     84694
DRUG_IN_IND_CODE                0
DRUG_IN_IND_TIERS           58444
DRUG_IN_IND_TIER1_AMOUNT    58492
DRUG_IN_IND_TIER2_AMOUNT    84918
TOT_IN_IND_CODE                 0
TOT_IN_IND_TIERS            26474
TOT_IN_IND_TIER1_AMOUNT     26474
TOT_IN_IND_TIER2_AMOUNT     84474
MED_OUT_IND_CODE                0
DRUG_OUT_IND_CODE               0
TOT_OUT_IND_CODE                0
MED_OUT_IND_AMOUNT          80229
DRUG_OUT_IND_AMOUNT         82434
TOT_OUT_IND_AMOUNT          75745
MED_IN_FAM_CODE                 0
MED_IN_FAM_TIERS            58478
MED_IN_FAM_TIER1_AMOUNT     58478
MED_IN_FAM_TIER2_AMOUNT     84694
DRUG_IN_FAM_CODE                0
DRUG_IN_FAM_TIERS           58478
DRUG_IN_FAM_TIER1_AMOUNT    58526
DRUG_IN_FAM_TIER2_AMOUNT    84918
TOT_IN_FAM_CODE                 0
TOT_IN_FAM_TIE

In [117]:
# As AMOUNT is supposed to be NUMERIC type, it is ok to have NaN values there
# However, TIERS is supposed to be INTEGER type.

# Values starting with MED_ or DRUG_ and ending with _TIERS have NaN values if value is integrated
# Values starting with TOT_ and ending with _TIERS have NaN values if value is separate as medical and drugs
# We code this as '99' if integrated and '999' if separated

# Identify all *_TIERS columns
tiers_cols = [col for col in deductibles.columns if col.endswith('_TIERS')]

# For TOT_*_TIERS columns: use 999 if missing (meaning separate)
tot_tiers_cols = [col for col in tiers_cols if col.startswith('TOT_')]
for col in tot_tiers_cols:
    deductibles[col] = deductibles[col].fillna(999).astype(int)

# For MED_*/DRUG_*_TIERS columns: use 99 if missing (meaning integrated)
meddrug_tiers_cols = [col for col in tiers_cols if col.startswith('MED_') or col.startswith('DRUG_')]
for col in meddrug_tiers_cols:
    deductibles[col] = deductibles[col].fillna(99).astype(int)

In [123]:
deductibles.head()

,PLANID,AREA,MED_IN_IND_CODE,MED_IN_IND_TIERS,MED_IN_IND_TIER1_AMOUNT,MED_IN_IND_TIER2_AMOUNT,DRUG_IN_IND_CODE,DRUG_IN_IND_TIERS,DRUG_IN_IND_TIER1_AMOUNT,DRUG_IN_IND_TIER2_AMOUNT,...,TOT_IN_FAM_CODE,TOT_IN_FAM_TIERS,TOT_IN_FAM_TIER1_AMOUNT,TOT_IN_FAM_TIER2_AMOUNT,MED_OUT_FAM_CODE,DRUG_OUT_FAM_CODE,TOT_OUT_FAM_CODE,MED_OUT_FAM_AMOUNT,DRUG_OUT_FAM_AMOUNT,TOT_OUT_FAM_AMOUNT
0,73836AK0950001,AK01,0,99,NaN,NaN,0,99,NaN,NaN,...,1,1,10000.0,NaN,0,0,1,NaN,NaN,35400.0
1,73836AK0950001,AK02,0,99,NaN,NaN,0,99,NaN,NaN,...,1,1,10000.0,NaN,0,0,1,NaN,NaN,35400.0
2,73836AK0930001,AK02,0,99,NaN,NaN,0,99,NaN,NaN,...,1,2,3000.0,6000.0,0,0,1,NaN,NaN,18000.0
3,73836AK0950001,AK03,0,99,NaN,NaN,0,99,NaN,NaN,...,1,1,10000.0,NaN,0,0,1,NaN,NaN,35400.0
4,73836AK0930001,AK03,0,99,NaN,NaN,0,99,NaN,NaN,...,1,2,3000.0,6000.0,0,0,1,NaN,NaN,18000.0


In [125]:
deductibles.to_csv('../data/normalized_tables/deductibles.csv', index=False)

In [127]:
# We see that all NaN values in the MOOP table are in columns that either end with the string '_TIERS' or '_AMOUNT'

moop.isnull().sum()

PLANID                          0
AREA                            0
MED_IN_IND_CODE                 0
MED_IN_IND_TIERS            84707
MED_IN_IND_TIER1_AMOUNT     84707
MED_IN_IND_TIER2_AMOUNT     84918
DRUG_IN_IND_CODE                0
DRUG_IN_IND_TIERS           84707
DRUG_IN_IND_TIER1_AMOUNT    84707
DRUG_IN_IND_TIER2_AMOUNT    84918
TOT_IN_IND_CODE                 0
TOT_IN_IND_TIERS              211
TOT_IN_IND_TIER1_AMOUNT       219
TOT_IN_IND_TIER2_AMOUNT     84628
MED_OUT_IND_CODE                0
DRUG_OUT_IND_CODE               0
TOT_OUT_IND_CODE                0
MED_OUT_IND_AMOUNT          84887
DRUG_OUT_IND_AMOUNT         84901
TOT_OUT_IND_AMOUNT          74424
MED_IN_FAM_CODE                 0
MED_IN_FAM_TIERS            84707
MED_IN_FAM_TIER1_AMOUNT     84707
MED_IN_FAM_TIER2_AMOUNT     84918
DRUG_IN_FAM_CODE                0
DRUG_IN_FAM_TIERS           84707
DRUG_IN_FAM_TIER1_AMOUNT    84707
DRUG_IN_FAM_TIER2_AMOUNT    84918
TOT_IN_FAM_CODE                 0
TOT_IN_FAM_TIE

In [129]:
# As AMOUNT is supposed to be NUMERIC type, it is ok to have NaN values there
# However, TIERS is supposed to be INTEGER type.

# Values starting with MED_ or DRUG_ and ending with _TIERS have NaN values if value is integrated
# Values starting with TOT_ and ending with _TIERS have NaN values if value is separate as medical and drugs
# We code this as '99' if integrated and '999' if separated

# Identify all *_TIERS columns
tiers_cols = [col for col in moop.columns if col.endswith('_TIERS')]

# For TOT_*_TIERS columns: use 999 if missing (meaning separate)
tot_tiers_cols = [col for col in tiers_cols if col.startswith('TOT_')]
for col in tot_tiers_cols:
    moop[col] = moop[col].fillna(999).astype(int)

# For MED_*/DRUG_*_TIERS columns: use 99 if missing (meaning integrated)
meddrug_tiers_cols = [col for col in tiers_cols if col.startswith('MED_') or col.startswith('DRUG_')]
for col in meddrug_tiers_cols:
    moop[col] = moop[col].fillna(99).astype(int)

In [131]:
moop.head()

,PLANID,AREA,MED_IN_IND_CODE,MED_IN_IND_TIERS,MED_IN_IND_TIER1_AMOUNT,MED_IN_IND_TIER2_AMOUNT,DRUG_IN_IND_CODE,DRUG_IN_IND_TIERS,DRUG_IN_IND_TIER1_AMOUNT,DRUG_IN_IND_TIER2_AMOUNT,...,TOT_IN_FAM_CODE,TOT_IN_FAM_TIERS,TOT_IN_FAM_TIER1_AMOUNT,TOT_IN_FAM_TIER2_AMOUNT,MED_OUT_FAM_CODE,DRUG_OUT_FAM_CODE,TOT_OUT_FAM_CODE,MED_OUT_FAM_AMOUNT,DRUG_OUT_FAM_AMOUNT,TOT_OUT_FAM_AMOUNT
0,73836AK0950001,AK01,0,99,NaN,NaN,0,99,NaN,NaN,...,1,1,16000.0,NaN,0,0,1,NaN,NaN,54600.0
1,73836AK0950001,AK02,0,99,NaN,NaN,0,99,NaN,NaN,...,1,1,16000.0,NaN,0,0,1,NaN,NaN,54600.0
2,73836AK0930001,AK02,0,99,NaN,NaN,0,99,NaN,NaN,...,1,1,12000.0,NaN,0,0,1,NaN,NaN,36000.0
3,73836AK0950001,AK03,0,99,NaN,NaN,0,99,NaN,NaN,...,1,1,16000.0,NaN,0,0,1,NaN,NaN,54600.0
4,73836AK0930001,AK03,0,99,NaN,NaN,0,99,NaN,NaN,...,1,1,12000.0,NaN,0,0,1,NaN,NaN,36000.0


In [133]:
moop.to_csv('../data/normalized_tables/moop.csv', index=False)

In [135]:
moop.columns

Index(['PLANID', 'AREA', 'MED_IN_IND_CODE', 'MED_IN_IND_TIERS',
       'MED_IN_IND_TIER1_AMOUNT', 'MED_IN_IND_TIER2_AMOUNT',
       'DRUG_IN_IND_CODE', 'DRUG_IN_IND_TIERS', 'DRUG_IN_IND_TIER1_AMOUNT',
       'DRUG_IN_IND_TIER2_AMOUNT', 'TOT_IN_IND_CODE', 'TOT_IN_IND_TIERS',
       'TOT_IN_IND_TIER1_AMOUNT', 'TOT_IN_IND_TIER2_AMOUNT',
       'MED_OUT_IND_CODE', 'DRUG_OUT_IND_CODE', 'TOT_OUT_IND_CODE',
       'MED_OUT_IND_AMOUNT', 'DRUG_OUT_IND_AMOUNT', 'TOT_OUT_IND_AMOUNT',
       'MED_IN_FAM_CODE', 'MED_IN_FAM_TIERS', 'MED_IN_FAM_TIER1_AMOUNT',
       'MED_IN_FAM_TIER2_AMOUNT', 'DRUG_IN_FAM_CODE', 'DRUG_IN_FAM_TIERS',
       'DRUG_IN_FAM_TIER1_AMOUNT', 'DRUG_IN_FAM_TIER2_AMOUNT',
       'TOT_IN_FAM_CODE', 'TOT_IN_FAM_TIERS', 'TOT_IN_FAM_TIER1_AMOUNT',
       'TOT_IN_FAM_TIER2_AMOUNT', 'MED_OUT_FAM_CODE', 'DRUG_OUT_FAM_CODE',
       'TOT_OUT_FAM_CODE', 'MED_OUT_FAM_AMOUNT', 'DRUG_OUT_FAM_AMOUNT',
       'TOT_OUT_FAM_AMOUNT'],
      dtype='object')

In [137]:
# There are two counties called 'Barbour County' in the US, one is in AL and one is in WV

county_area[county_area['COUNTY'] == 'Barbour County']

,FIPS,COUNTY,AREA_COUNT,AREA
2,1005,Barbour County,1,AL13
3017,54001,Barbour County,1,WV09
